In [ ]:
#import logging
import os, time, json
from typing import Any
from azure.ai.projects import AIProjectClient  
from azure.identity import DefaultAzureCredential  
from azure.ai.projects.models import SubmitToolOutputsAction, RequiredFunctionToolCall, ToolSet, FunctionTool
from weather_functions import weather_functions, get_current_weather, get_lat_lon
from IPython.display import Markdown, display

# Set up logging
#logging.basicConfig(level=logging.INFO)

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

with project_client:
    functions = FunctionTool(weather_functions)
    print(functions.definitions)
    toolset = ToolSet()
    toolset.add(functions)

    # Create an agent and run user's request with function calls
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="weather-agent",
        instructions="You are a helpful assistant that provides weather information.",
        toolset=toolset,
    )
    print(f"Created agent, ID: {agent.id}")

    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What's the current weather information for New York?",
    )
    print(f"Created message, ID: {message.id}")

    run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        # Extract the function name and arguments from the tool call
                        operation_id = tool_call.function.name
                        arguments = json.loads(tool_call.function.arguments)
                        print(f"Handling tool call: {operation_id} with arguments: {arguments}")

                        # Handle the specific operation
                        if operation_id == "get_current_weather":
                            lat = arguments.get("lat")
                            lon = arguments.get("lon")

                            # If lat and lon are not provided, geocode the location
                            if not lat or not lon:
                                location_name = arguments.get('q') or arguments.get('location')
                                if not location_name:
                                    raise Exception("Location name not provided.")
                                location_data = get_lat_lon(location_name)
                                if not location_data:
                                    raise Exception(f"Could not find location: {location_name}")
                                lat = location_data['lat']
                                lon = location_data['lon']
                            
                            # Call the get_current_weather function
                            weather_data = get_current_weather(float(lat), float(lon))
                            if not weather_data:
                                raise Exception("Could not retrieve weather data.")
                            
                            # Prepare the result content with additional fields
                            try:
                                description = weather_data['weather'][0]['description'] if weather_data.get('weather') else "No description available."
                                temp = weather_data.get('temp', 'N/A')
                                feels_like = weather_data.get('feels_like', 'N/A')
                                pressure = weather_data.get('pressure', 'N/A')
                                humidity = weather_data.get('humidity', 'N/A')
                                wind_speed = weather_data.get('wind_speed', 'N/A')
                                weather_id = weather_data['weather'][0]['id'] if weather_data.get('weather') else 'N/A'

                                result_content = {
                                    "lat": lat,
                                    "lon": lon,
                                    "description": description,
                                    "temperature": temp,
                                    "feels_like": feels_like,
                                    "pressure": pressure,
                                    "humidity": humidity,
                                    "wind_speed": wind_speed,
                                    "weather_id": weather_id
                                }
                            except (IndexError, KeyError) as e:
                                raise Exception(f"Error parsing weather data: {e}")

                            # Append the tool output as a JSON string
                            tool_outputs.append({
                                "tool_call_id": tool_call.id,
                                "output": json.dumps({
                                    "status": "success",
                                    "content": result_content
                                })
                            })

                        elif operation_id == "get_lat_lon":
                            location = arguments.get("location")
                            if not location:
                                raise Exception("Location not provided.")

                            # Call the get_lat_lon function
                            location_data = get_lat_lon(location)
                            if not location_data:
                                raise Exception(f"Could not geocode location: {location}")

                            # Prepare the result content
                            result_content = location_data

                            # Append the tool output as a JSON string
                            tool_outputs.append({
                                "tool_call_id": tool_call.id,
                                "output": json.dumps({
                                    "status": "success",
                                    "content": result_content
                                })
                            })

                        else:
                            # If operation_id is unrecognized
                            raise Exception(f"Unknown operation_id: {operation_id}")
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")
                        # Handle exceptions and provide error output as a JSON string
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": json.dumps({
                                "status": "error",
                                "message": str(e)
                            })
                        })

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.agents.submit_tool_outputs_to_run(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Convert messages.data list into a dictionary
    messages_json = json.dumps(messages.data[0].as_dict())

    messages_dict = messages.data[0]

    # Print the value
    display(Markdown(messages_dict.content[0].text['value']))

    # Print formatted json
    print(json.dumps(json.loads(messages_json), indent=4, sort_keys=True))